In [12]:
import requests
from bs4 import BeautifulSoup
import re
import json
from selenium import webdriver
from lxml import etree
import os
from os import path
import shutil
import time
import demjson

In [13]:
def getResponse(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'
    }
    response = requests.get(url=url, headers=headers)
    response.encoding = 'utf-8'
    return response

In [14]:
def getResponseNotForText(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'
    }
    response = requests.get(url=url, headers=headers)
    #response.encoding = 'utf-8'
    return response

In [2]:
# http://tv.cctv.com/lm/zhyy?spm=C28340.PlFTqGe6Zk8M.S70924.13
# https://tv.cctv.com/2020/03/09/VIDElK1NAIGnDfE1sIpSwg1L200309.shtml?spm=C45305.P925SXLfdI0z.EVHTEeaRtdRD.7
# 先解决通过流媒体播放的视频下载
# m3u8文件+ts文件
# 无法通过视频地址直接获取m3u8地址 需要使用selenium

In [19]:
# 测试是否可以直接获取链接
url = 'https://tv.cctv.com/2020/03/09/VIDElK1NAIGnDfE1sIpSwg1L200309.shtml?spm=C45305.P925SXLfdI0z.EVHTEeaRtdRD.7'
page_text = getResponse(url).text
print(page_text)
# 测试结果：需要selenium 

                <!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
	<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
	
	<title>《中华医药 抗击疫情》 20200306 北京战“疫”共盼春来_CCTV节目官网-CCTV-4_央视网(cctv.com)</title>
	<meta name=keywords content="中华医药 抗击疫情，李七月，屠志涛，北京，确诊患者，中西医结合疗法">
	<meta name=description content="本期节目主要内容：
    北京作为我国的首都，抗击疫情保障民众安全不能有丝毫松懈。面对疫情，北京相关部门迅速行动，成功防止了疫情的蔓延。北京首例确诊患者在中西医结合疗法的治疗下成功康复，中医名师深夜会诊，危重患者转危为安。本期节目将带您了解北京的疫情发展以及防控的过程。
（《中华医药 抗击疫情》 20200306 北京战“疫”共盼春来）">
	<meta name="spm-id" content="C52507945305">
	<meta property="og:title" content="《中华医药 抗击疫情》 20200306 北京战“疫”共盼春来" />
	<meta property="og:image" content="//p1.img.cctvpic.com/fmspic/2020/03/09/5815f917075e4dd6a8fa49a0b7609197-912.jpg" />
	<meta property="og:description" content="本期节目主要内容：
    北京作为我国的首都，抗击疫情保障民众安全不能有丝毫松懈。面对疫情，北京相关部门迅速行动，成功防止了疫情的蔓延。北京首例确诊患者在中西医结合疗法的治疗下成功康复，中医名师深夜会诊，危重患者转危

In [18]:
url = 'https://tv.cctv.com/2020/03/09/VIDElK1NAIGnDfE1sIpSwg1L200309.shtml?spm=C45305.P925SXLfdI0z.EVHTEeaRtdRD.7'
# 需要模拟
browser = webdriver.Chrome()
browser.get(url)
html = browser.page_source
print(html)

<html xmlns="http://www.w3.org/1999/xhtml"><head><style class="vjs-styles-defaults">
      .video-js {
        width: 300px;
        height: 150px;
      }

      .vjs-fluid {
        padding-top: 56.25%
      }
    </style><style class="vjs-styles-dimensions">
      .myFlash_player-dimensions {
        width: 1280px;
        height: 720px;
      }

      .myFlash_player-dimensions.vjs-fluid {
        padding-top: 56.25%;
      }
    </style>
	<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
	
	<title>《中华医药 抗击疫情》 20200306 北京战“疫”共盼春来_CCTV节目官网-CCTV-4_央视网(cctv.com)</title>
	<meta name="keywords" content="中华医药 抗击疫情，李七月，屠志涛，北京，确诊患者，中西医结合疗法">
	<meta name="description" content="本期节目主要内容：
    北京作为我国的首都，抗击疫情保障民众安全不能有丝毫松懈。面对疫情，北京相关部门迅速行动，成功防止了疫情的蔓延。北京首例确诊患者在中西医结合疗法的治疗下成功康复，中医名师深夜会诊，危重患者转危为安。本期节目将带您了解北京的疫情发展以及防控的过程。
（《中华医药 抗击疫情》 20200306 北京战“疫”共盼春来）">
	<meta name="spm-id" content="C52507945305">
	<meta property="og:title" content="《中华医药 抗击疫情》 20200306 北京战“疫”共盼春来">
	<meta proper

In [21]:
tree = etree.HTML(html)
sourseUrl = tree.xpath('//*[@id="myFlash_player_html5_api"]/source/@src')[0]
print(sourseUrl)

https://dh5.cntv.kcdnvip.com/asp/h5e/hls/main/0303000a/3/default/5815f917075e4dd6a8fa49a0b7609197/main.m3u8?maxbr=2048&contentid=15120519184043


In [15]:
# 获取m3u8文件地址
def getM3U8(url):
    browser = webdriver.Chrome()
    browser.get(url)
    html = browser.page_source
    tree = etree.HTML(html)
    sourseUrl = tree.xpath('//*[@id="myFlash_player_html5_api"]/source/@src')
    if len(sourseUrl) == 0:
        return ""
    return sourseUrl[0]

In [24]:
print(getM3U8('https://tv.cctv.com/2019/09/07/VIDER1UBCcaEIymg11ol5Xjz190907.shtml?spm=C45305.P925SXLfdI0z.EVHTEeaRtdRD.26'))

https://dh5.cntv.kcdnvip.com/asp/h5e/hls/main/0303000a/3/default/e99c3c9c56464695b148c4d69bebf55b/main.m3u8?maxbr=2048&contentid=15120519184043


In [43]:
#  https://blog.csdn.net/liujiayu2/article/details/86083400
# 下载M3U8文件中所有片段
# https://dh5.cntv.kcdnvip.com/asp/h5e/hls/1200/0303000a/3/default/e99c3c9c56464695b148c4d69bebf55b/1200.m3u8
# 需要通过视频页面的m3u8地址下载到的m3u8文件获取事及m3u8文件
url = 'https://dh5.cntv.kcdnvip.com/asp/h5e/hls/main/0303000a/3/default/e99c3c9c56464695b148c4d69bebf55b/main.m3u8?maxbr=2048&contentid=15120519184043'
downloadPath = os.getcwd() + '/download'
if not os.path.exists(downloadPath):
    os.mkdir(downloadPath)
allContent = getResponse(url).text# 获取M3U8的文件内容
fileLine = allContent.split('\n')# 读取文件里的每一行
#print(fileLine)
m3u8Url = 'https://dh5.cntv.kcdnvip.com' + fileLine[-4]
print(m3u8Url)
allContent = getResponse(m3u8Url).text
fileLine = allContent.split('\n')

https://dh5.cntv.kcdnvip.com/asp/h5e/hls/1200/0303000a/3/default/e99c3c9c56464695b148c4d69bebf55b/1200.m3u8


In [45]:
# 通过判断文件头来确定是否是M3U8文件
if fileLine[0] != "#EXTM3U":
    raise BaseException(u"非M3U8的链接")
else:
    unknow = True# 用来判断是否找到了下载地址
    for index, line in enumerate(fileLine):
        if "EXTINF" in line:
            unknow = False
            # 拼出ts片段的URL
            # https://dh5.cntv.kcdnvip.com/asp/h5e/hls/1200/0303000a/3/default/e99c3c9c56464695b148c4d69bebf55b/16.ts
            tsUrl = m3u8Url.rsplit('/', 1)[0] + '/' + fileLine[index + 1]# i.ts在下一行
            #print(tsUrl)
            res = getResponseNotForText(tsUrl)
            cFuleName = str(fileLine[index + 1])
            with open(downloadPath + '/' + cFuleName, 'ab') as fp:
                fp.write(res.content)
                fp.flush()
    if unknow:
        raise BaseException(u"未找到对应的下载地址")
    else:
        print("下载完成")

下载完成


In [29]:
def downloadTS(url):
    #  https://blog.csdn.net/liujiayu2/article/details/86083400
    # 下载M3U8文件中所有片段
    # https://dh5.cntv.kcdnvip.com/asp/h5e/hls/1200/0303000a/3/default/e99c3c9c56464695b148c4d69bebf55b/1200.m3u8
    # 需要通过视频页面的m3u8地址下载到的m3u8文件获取事及m3u8文件
    #url = 'https://dh5.cntv.kcdnvip.com/asp/h5e/hls/main/0303000a/3/default/e99c3c9c56464695b148c4d69bebf55b/main.m3u8?maxbr=2048&contentid=15120519184043'
    url = getM3U8(url)# 获取main M3U8 的url
    if url == "":
        print("error")
        return
    print('main M3U8: ', url)
    downloadPath = os.getcwd() + '/download'
    if not os.path.exists(downloadPath):
        os.mkdir(downloadPath)
    allContent = getResponse(url).text# 获取M3U8的文件内容
    fileLine = allContent.split('\n')# 读取文件里的每一行
    print(fileLine)
    m3u8Url = 'https://dh5.cntv.kcdnvip.com' + fileLine[-2]# 实际视频的M3U8的URL
    print("实际M3U8: ", m3u8Url)
    allContent = getResponse(m3u8Url).text
    fileLine = allContent.split('\n')
    
    # 通过判断文件头来确定是否是M3U8文件
    if fileLine[0] != "#EXTM3U":
        raise BaseException(u"非M3U8的链接")
    else:
        unknow = True# 用来判断是否找到了下载地址
        for index, line in enumerate(fileLine):
            if "EXTINF" in line:
                unknow = False
                # 拼出ts片段的URL
                # https://dh5.cntv.kcdnvip.com/asp/h5e/hls/1200/0303000a/3/default/e99c3c9c56464695b148c4d69bebf55b/16.ts
                tsUrl = m3u8Url.rsplit('/', 1)[0] + '/' + fileLine[index + 1]# i.ts在下一行
                #print(tsUrl)
                res = getResponseNotForText(tsUrl)
                cFuleName = str(fileLine[index + 1])
                with open(downloadPath + '/' + cFuleName, 'ab') as fp:
                    fp.write(res.content)
                    fp.flush()
        if unknow:
            raise BaseException(u"未找到对应的下载地址")
        else:
            print("ts 下载完成")

In [23]:
# ts文件合并
def file_walker(path):
    file_list = []
    for root, dirs, files in os.walk(path): # 生成器
        for fn in files:
            p = str(root+'/'+fn)
            file_list.append(p)
 
    #print(file_list)
    return file_list
 
def combine(ts_path, combine_path, file_name):
    file_list = file_walker(ts_path)
    file_path = combine_path + file_name + '.ts'
    with open(file_path, 'wb+') as fw:
        for i in range(len(file_list)): 
            fw.write(open(file_list[i], 'rb').read())

In [47]:
combine('./download', 'F:/work/cctv/中华医药/', 'haha')

['./download/0.ts', './download/1.ts', './download/10.ts', './download/100.ts', './download/101.ts', './download/102.ts', './download/103.ts', './download/104.ts', './download/105.ts', './download/106.ts', './download/107.ts', './download/108.ts', './download/109.ts', './download/11.ts', './download/110.ts', './download/111.ts', './download/112.ts', './download/113.ts', './download/114.ts', './download/115.ts', './download/116.ts', './download/117.ts', './download/118.ts', './download/119.ts', './download/12.ts', './download/120.ts', './download/121.ts', './download/122.ts', './download/123.ts', './download/124.ts', './download/125.ts', './download/126.ts', './download/127.ts', './download/128.ts', './download/129.ts', './download/13.ts', './download/130.ts', './download/131.ts', './download/132.ts', './download/133.ts', './download/134.ts', './download/135.ts', './download/136.ts', './download/137.ts', './download/138.ts', './download/139.ts', './download/14.ts', './download/140.ts', 

In [24]:
# 下载并合成视频
def downloadAndCombine(url):
    fileName = (url.split('/')[-1]).split('.')[0]
    downloadTS(url)
    combine('./download', 'F:/work/cctv/中华医药/video/', fileName)
    print(fileName, " successful")

In [59]:
# 'https://tv.cctv.com/2019/09/07/VIDER1UBCcaEIymg11ol5Xjz190907.shtml?spm=C45305.P925SXLfdI0z.EVHTEeaRtdRD.26'
url = 'https://tv.cctv.com/2019/09/07/VIDER1UBCcaEIymg11ol5Xjz190907.shtml?spm=C45305.P925SXLfdI0z.EVHTEeaRtdRD.26'
downloadAndCombine(url)

ts 下载完成
VIDER1UBCcaEIymg11ol5Xjz190907  successful


In [18]:
# 删除之前下载的一段段的ts文件
def clear():
    shutil.rmtree('./download')
    time.sleep(1)
    os.mkdir('download')

In [60]:
clear()

In [62]:
# 获取每页的所有视频链接
# http://api.cntv.cn/NewVideo/getVideoListByColumn?id=TOPC1451541666791291&n=15&sort=desc&p=3&mode=0&serviceId=tvcctv&cb=Callback
# http://api.cntv.cn/NewVideo/getVideoListByColumn?id=TOPC1451541666791291&n=15&sort=desc&p=6&mode=0&serviceId=tvcctv&cb=Callback
# https:\/\/tv.cctv.com\/2017\/11\/19\/VIDEv1E495mgQnw4tYab8xxx171119.shtml
# https://tv.cctv.com/2017/11/19/VIDEv1E495mgQnw4tYab8xxx171119.shtml

In [72]:
url = 'http://api.cntv.cn/NewVideo/getVideoListByColumn?id=TOPC1451541666791291&n=15&sort=desc&p=3&mode=0&serviceId=tvcctv&cb=Callback'
page_text = getResponse(url).text
print(page_text)

Callback({"data":{"total":577,"list":[{"guid":"dfefb22290094e50afa2e6a1ee5c9f98","id":"VIDERgPUv2KNZfK8LRtclRB8190406","time":"2019-04-06 17:15:00","title":"\u300a\u4e2d\u534e\u533b\u836f\u300b 20190406 \u5e26\u7624\u751f\u5b58 \u4e0e\u764c\u5171\u821e","length":"00:40:54","image":"http:\/\/p1.img.cctvpic.com\/fmspic\/2019\/04\/06\/dfefb22290094e50afa2e6a1ee5c9f98-1210.jpg","focus_date":1554547747870,"brief":"\u672c\u671f\u8282\u76ee\u4e3b\u8981\u5185\u5bb9\uff1a  \u8eab\u60a3\u4e24\u79cd\u764c\u75c7\u5374\u5e26\u7624\u751f\u5b58\u4e86\u5341\u4e94\u5e74\uff0c\u80bf\u7624\u9762\u524d\uff0c\u5207\u8fd8\u662f\u4e0d\u5207\uff1f\u4e0d\u540c\u7684\u533b\u9662\u6709\u4e0d\u540c\u7684\u8bca\u7597\u7ed3\u679c\uff0c\u53c8\u8be5\u600e\u4e48\u529e\uff1f\u656c\u8bf7\u6536\u770b\u672c\u671f\u8282\u76ee\u3002\uff08\u300a\u4e2d\u534e\u533b\u836f\u300b 20190406 \u5e26\u7624\u751f\u5b58 \u4e0e\u764c\u5171\u821e\uff09","url":"https:\/\/tv.cctv.com\/2019\/04\/06\/VIDERgPUv2KNZfK8LRtclRB8190406.shtml"},{"g

In [76]:
jsonObj = demjson.decode(page_text[9:-2])
print(jsonObj)

{'data': {'total': 577, 'list': [{'guid': 'dfefb22290094e50afa2e6a1ee5c9f98', 'id': 'VIDERgPUv2KNZfK8LRtclRB8190406', 'time': '2019-04-06 17:15:00', 'title': '《中华医药》 20190406 带瘤生存 与癌共舞', 'length': '00:40:54', 'image': 'http://p1.img.cctvpic.com/fmspic/2019/04/06/dfefb22290094e50afa2e6a1ee5c9f98-1210.jpg', 'focus_date': 1554547747870, 'brief': '本期节目主要内容：  身患两种癌症却带瘤生存了十五年，肿瘤面前，切还是不切？不同的医院有不同的诊疗结果，又该怎么办？敬请收看本期节目。（《中华医药》 20190406 带瘤生存 与癌共舞）', 'url': 'https://tv.cctv.com/2019/04/06/VIDERgPUv2KNZfK8LRtclRB8190406.shtml'}, {'guid': 'f0dccc4c23204e49ac4b6b362a724794', 'id': 'VIDE0kk2TM6C0H6lGsXWoFze190330', 'time': '2019-03-30 17:15:00', 'title': '《中华医药》 20190330 无“胃”生死', 'length': '00:41:03', 'image': 'http://p1.img.cctvpic.com/fmspic/2019/03/30/f0dccc4c23204e49ac4b6b362a724794-1210.jpg', 'focus_date': 1553941998992, 'brief': '本期节目主要内容： 有一种疾病，全人类都害怕，它就是癌症。到目前为止，人类对癌症还没有找到特别有把握的治疗方法，癌症患者和他的家人往往承受着巨大的心理压力。（《中华医药》 20190330 无“胃”生死）', 'url': 'https://tv.cctv.com/2019/03/30/VIDE0kk2TM6C0H6lGsXWoFze

In [25]:
# # 获取每页的所有视频链接
# 使用demjson
# 涉及将 application/x-javascript转为json
def getVideoUrls(url):
    page_text = getResponse(url).text
    jsonObj = demjson.decode(page_text[9:-2])
    linkList = []
    for obj in jsonObj['data']['list']:
        linkList.append(obj['url'])
    return linkList

In [80]:
for link in getVideoUrls('http://api.cntv.cn/NewVideo/getVideoListByColumn?id=TOPC1451541666791291&n=15&sort=desc&p=1&mode=0&serviceId=tvcctv&cb=Callback'):
    print(link)

https://tv.cctv.com/2020/03/21/VIDEGaFjYnBPTHt5UW3f0wi5200321.shtml
https://tv.cctv.com/2020/03/15/VIDEHeVmDbnAJS97Xnx34aXH200315.shtml
https://tv.cctv.com/2020/03/09/VIDElK1NAIGnDfE1sIpSwg1L200309.shtml
https://tv.cctv.com/2020/03/09/VIDEVFJ1fhXzkjzEhpYoxx6X200309.shtml
https://tv.cctv.com/2020/03/05/VIDEojsWIebFPlpQXYmrActs200305.shtml
https://tv.cctv.com/2020/03/05/VIDEWC9Fy4H54bPY070CTOg7200305.shtml
https://tv.cctv.com/2020/03/05/VIDEngA28AHNJyDYsLm5A3EH200305.shtml
https://tv.cctv.com/2019/09/14/VIDEuCPXZwZV9Q7YOuwohPot190914.shtml
https://tv.cctv.com/2019/09/07/VIDER1UBCcaEIymg11ol5Xjz190907.shtml
https://tv.cctv.com/2019/08/31/VIDEgptDnyFOK9PRPH9OdtYf190831.shtml
https://tv.cctv.com/2019/08/24/VIDEgiHasu8iIiWFNJIDranu190824.shtml
https://tv.cctv.com/2019/08/17/VIDE9JKXjQDkk4fiY3gVqeS2190817.shtml
https://tv.cctv.com/2019/08/10/VIDEubng7yqCpSLbnYHLUb0a190810.shtml
https://tv.cctv.com/2019/08/03/VIDEiKWKleH67CnRL21bCz6b190803.shtml
https://tv.cctv.com/2019/07/27/VIDElRU8iUuJT2hSu

In [30]:
for i in range(32, 40):
    url = f'http://api.cntv.cn/NewVideo/getVideoListByColumn?id=TOPC1451541666791291&n=15&sort=desc&p={i}&mode=0&serviceId=tvcctv&cb=Callback'
    print(f'页面 {i}', url)
    #cnt = 0
    for link in getVideoUrls(url):
        #if i == 32 and cnt < 2:
            #continue
        cnt = cnt + 1
        print('开始下载：', link)
        downloadAndCombine(link)
        clear()

页面 32 http://api.cntv.cn/NewVideo/getVideoListByColumn?id=TOPC1451541666791291&n=15&sort=desc&p=32&mode=0&serviceId=tvcctv&cb=Callback
开始下载： https://tv.cctv.com/2010/11/04/VIDE1356351038737683.shtml
main M3U8:  https://dh5.cntv.qcloudcdn.com/asp/h5e/hls/main/0303000a/3/default/8c33846b438e43514ccdc6a2bcd37ce0/main.m3u8?maxbr=2048&contentid=15120519184043
['#EXTM3U', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=460800, RESOLUTION=480x360', '/asp/h5e/hls/450/0303000a/3/default/8c33846b438e43514ccdc6a2bcd37ce0/450.m3u8', '']
实际M3U8:  https://dh5.cntv.kcdnvip.com/asp/h5e/hls/450/0303000a/3/default/8c33846b438e43514ccdc6a2bcd37ce0/450.m3u8
ts 下载完成
VIDE1356351038737683  successful
开始下载： https://tv.cctv.com/2010/10/28/VIDE1356350955852793.shtml
main M3U8:  https://dh5.cntv.lxdns.com/asp/h5e/hls/main/0303000a/3/default/0f49f3b18334495d1f4a1788f692f5c9/main.m3u8?maxbr=2048&contentid=15120519184043
['#EXTM3U', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=460800, RESOLUTION=480x360', '/asp/h5e/hls/45

ts 下载完成
VIDE1356351030985606  successful
开始下载： https://tv.cctv.com/2010/07/08/VIDE1356351037694674.shtml
main M3U8:  https://dh5.cntv.qcloudcdn.com/asp/h5e/hls/main/0303000a/3/default/152e84bf22da4865094000ac12821aaf/main.m3u8?maxbr=2048&contentid=15120519184043
['#EXTM3U', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=460800, RESOLUTION=480x360', '/asp/h5e/hls/450/0303000a/3/default/152e84bf22da4865094000ac12821aaf/450.m3u8', '']
实际M3U8:  https://dh5.cntv.kcdnvip.com/asp/h5e/hls/450/0303000a/3/default/152e84bf22da4865094000ac12821aaf/450.m3u8
ts 下载完成
VIDE1356351037694674  successful
开始下载： https://tv.cctv.com/2010/07/01/VIDE1356351255108896.shtml
main M3U8:  https://dh5.cntv.qcloudcdn.com/asp/h5e/hls/main/0303000a/3/default/0c7b2dcec78f482896d86690ca6c5866/main.m3u8?maxbr=2048&contentid=15120519184043
['#EXTM3U', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=460800, RESOLUTION=480x360', '/asp/h5e/hls/450/0303000a/3/default/0c7b2dcec78f482896d86690ca6c5866/450.m3u8', '']
实际M3U8:  https://dh5.

main M3U8:  https://dh5.cntv.baishancdnx.cn/asp/h5e/hls/main/0303000a/3/default/d09941a234de46e91ada8596acb837c0/main.m3u8?maxbr=2048&contentid=15120519184043
['#EXTM3U', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=460800, RESOLUTION=480x360', '/asp/h5e/hls/450/0303000a/3/default/d09941a234de46e91ada8596acb837c0/450.m3u8', '']
实际M3U8:  https://dh5.cntv.kcdnvip.com/asp/h5e/hls/450/0303000a/3/default/d09941a234de46e91ada8596acb837c0/450.m3u8
ts 下载完成
VIDE1356350879872998  successful
开始下载： https://tv.cctv.com/2010/04/24/VIDE1356350799905945.shtml
main M3U8:  https://dh5.cntv.baishancdnx.cn/asp/h5e/hls/main/0303000a/3/default/7588c639f82142919ead49a6205f7a50/main.m3u8?maxbr=2048&contentid=15120519184043
['#EXTM3U', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=460800, RESOLUTION=480x360', '/asp/h5e/hls/450/0303000a/3/default/7588c639f82142919ead49a6205f7a50/450.m3u8', '']
实际M3U8:  https://dh5.cntv.kcdnvip.com/asp/h5e/hls/450/0303000a/3/default/7588c639f82142919ead49a6205f7a50/450.m3u8
ts 下载完成
V

ts 下载完成
VIDE1356350922088467  successful
开始下载： https://tv.cctv.com/2010/04/24/VIDE1356351274357174.shtml
main M3U8:  https://dh5.cntv.qcloudcdn.com/asp/h5e/hls/main/0303000a/3/default/edffcd65573542d7f23cc4b251657b9d/main.m3u8?maxbr=2048&contentid=15120519184043
['#EXTM3U', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=460800, RESOLUTION=480x360', '/asp/h5e/hls/450/0303000a/3/default/edffcd65573542d7f23cc4b251657b9d/450.m3u8', '']
实际M3U8:  https://dh5.cntv.kcdnvip.com/asp/h5e/hls/450/0303000a/3/default/edffcd65573542d7f23cc4b251657b9d/450.m3u8
ts 下载完成
VIDE1356351274357174  successful
开始下载： https://tv.cctv.com/2010/04/24/VIDE1356350895461248.shtml
main M3U8:  https://dh5.cntv.qcloudcdn.com/asp/h5e/hls/main/0303000a/3/default/e4620b6946154f96908769b32f562135/main.m3u8?maxbr=2048&contentid=15120519184043
['#EXTM3U', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=460800, RESOLUTION=480x360', '/asp/h5e/hls/450/0303000a/3/default/e4620b6946154f96908769b32f562135/450.m3u8', '']
实际M3U8:  https://dh5.

ts 下载完成
VIDE1356350881683116  successful
开始下载： https://tv.cctv.com/2010/04/13/VIDE1356350821950917.shtml
main M3U8:  https://dh5.cntv.qcloudcdn.com/asp/h5e/hls/main/0303000a/3/default/a64822a2ead844c6dd3554b0f936e655/main.m3u8?maxbr=2048&contentid=15120519184043
['#EXTM3U', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=460800, RESOLUTION=480x360', '/asp/h5e/hls/450/0303000a/3/default/a64822a2ead844c6dd3554b0f936e655/450.m3u8', '']
实际M3U8:  https://dh5.cntv.kcdnvip.com/asp/h5e/hls/450/0303000a/3/default/a64822a2ead844c6dd3554b0f936e655/450.m3u8
ts 下载完成
VIDE1356350821950917  successful
开始下载： https://tv.cctv.com/2010/04/13/VIDE1356350836166729.shtml
main M3U8:  https://dh5.cntv.baishancdnx.cn/asp/h5e/hls/main/0303000a/3/default/b97353aa5ae243c82a15eca1d321c84d/main.m3u8?maxbr=2048&contentid=15120519184043
['#EXTM3U', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=460800, RESOLUTION=480x360', '/asp/h5e/hls/450/0303000a/3/default/b97353aa5ae243c82a15eca1d321c84d/450.m3u8', '']
实际M3U8:  https://dh5

ts 下载完成
VIDE1356350807889306  successful
开始下载： https://tv.cctv.com/2010/01/21/VIDE1356350917670428.shtml
main M3U8:  https://dh5.cntv.kcdnvip.com/asp/h5e/hls/main/0303000a/3/default/7b11e5713f9d4fe3f45ef0a66d964891/main.m3u8?maxbr=2048&contentid=15120519184043
['#EXTM3U', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=460800, RESOLUTION=480x360', '/asp/h5e/hls/450/0303000a/3/default/7b11e5713f9d4fe3f45ef0a66d964891/450.m3u8', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=870400, RESOLUTION=480x360', '/asp/h5e/hls/850/0303000a/3/default/7b11e5713f9d4fe3f45ef0a66d964891/850.m3u8', '']
实际M3U8:  https://dh5.cntv.kcdnvip.com/asp/h5e/hls/850/0303000a/3/default/7b11e5713f9d4fe3f45ef0a66d964891/850.m3u8
ts 下载完成
VIDE1356350917670428  successful
开始下载： https://tv.cctv.com/2010/01/14/VIDE1356351265320990.shtml
main M3U8:  https://dh5.cntv.kcdnvip.com/asp/h5e/hls/main/0303000a/3/default/35d6b70121514b37b46d5cbc8fb7586f/main.m3u8?maxbr=2048&contentid=15120519184043
['#EXTM3U', '#EXT-X-STREAM-INF:PROGRAM-ID

ts 下载完成
VIDE1356351124432569  successful
开始下载： https://tv.cctv.com/2009/11/28/VIDE1356350925524495.shtml
main M3U8:  https://dh5.cntv.kcdnvip.com/asp/h5e/hls/main/0303000a/3/default/7169357711c5432465b37f8c0f54b69e/main.m3u8?maxbr=2048&contentid=15120519184043
['#EXTM3U', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=460800, RESOLUTION=480x360', '/asp/h5e/hls/450/0303000a/3/default/7169357711c5432465b37f8c0f54b69e/450.m3u8', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=870400, RESOLUTION=480x360', '/asp/h5e/hls/850/0303000a/3/default/7169357711c5432465b37f8c0f54b69e/850.m3u8', '']
实际M3U8:  https://dh5.cntv.kcdnvip.com/asp/h5e/hls/850/0303000a/3/default/7169357711c5432465b37f8c0f54b69e/850.m3u8
ts 下载完成
VIDE1356350925524495  successful
开始下载： https://tv.cctv.com/2009/11/28/VIDE1356351089580268.shtml
main M3U8:  https://dh5.cntv.qcloudcdn.com/asp/h5e/hls/main/0303000a/3/default/d044d7948b6948487bbce7b1846ba849/main.m3u8?maxbr=2048&contentid=15120519184043
['#EXTM3U', '#EXT-X-STREAM-INF:PROGRAM-

ts 下载完成
VIDE1356351249895847  successful
开始下载： https://tv.cctv.com/2009/08/27/VIDE1356350943481662.shtml
main M3U8:  https://dh5.cntv.kcdnvip.com/asp/h5e/hls/main/0303000a/3/default/acd877e57fa04c2d7e797bb2358581af/main.m3u8?maxbr=2048&contentid=15120519184043
['#EXTM3U', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=460800, RESOLUTION=480x360', '/asp/h5e/hls/450/0303000a/3/default/acd877e57fa04c2d7e797bb2358581af/450.m3u8', '#EXT-X-STREAM-INF:PROGRAM-ID=1, BANDWIDTH=870400, RESOLUTION=480x360', '/asp/h5e/hls/850/0303000a/3/default/acd877e57fa04c2d7e797bb2358581af/850.m3u8', '']
实际M3U8:  https://dh5.cntv.kcdnvip.com/asp/h5e/hls/850/0303000a/3/default/acd877e57fa04c2d7e797bb2358581af/850.m3u8
ts 下载完成
VIDE1356350943481662  successful
页面 39 http://api.cntv.cn/NewVideo/getVideoListByColumn?id=TOPC1451541666791291&n=15&sort=desc&p=39&mode=0&serviceId=tvcctv&cb=Callback
开始下载： https://tv.cctv.com/2009/08/17/VIDE1356351142968756.shtml
main M3U8:  https://dh5.cntv.qcloudcdn.com/asp/h5e/hls/main/03

In [ ]:
# 缺
# 2-13 2-14 2-15
# 11-13 11-14 11-15
# 14-all